## Import libraries

In [ ]:
import ee
import geemap
import os, re, json, time
import geopandas as gpd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, mapping

## Start Connection with Google Earth Engine

In [ ]:
ee.Authenticate(force=True)
ee.Initialize()
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## Set up environmental variables

In [ ]:
root_folder = os.getcwd().split('\\')
root_folder = root_folder[1:-2]
if root_folder[-1] != 'fireRunSeverity':
    print("Didn't get correct root folder!")
    print(root_folder)
root_folder = os.path.join('C:\\', *root_folder)

in_Name = 'GIF14_Au'
root_folder

# local folder
dataFLD = os.path.join(root_folder, 'data', 'fireruns', in_Name)
inFLD = os.path.join(dataFLD, 'input')

## Download to local folder (TBD)

In [ ]:
# Check Folder
outGEEFLD = os.path.join('root_folder', 'data', 'GEE', in_Name)
if os.path.isdir(outGEEFLD):
    pass
else:
    os.mkdir(outGEEFLD)


False